# Lab 6.2: Linear Regression

In [1]:
%pylab inline

import pandas as pd
import statsmodels.api as sm
import yaml

from seaborn import pairplot
from sqlalchemy import create_engine

pg_creds = yaml.load(open('../../pg_creds.yaml'))['student']

engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**pg_creds))

Populating the interactive namespace from numpy and matplotlib


**Question 1**  

Using the cars data,

1) Fit a simple linear regression to predict `mpg` using `weight`.  

In [2]:
cars = pd.read_sql("SELECT * FROM cars WHERE horsepower IS NOT NULL;", engine, index_col='index')

In [3]:
cars.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model,origin,car_name
index,,,,,,,,,
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


In [4]:
X = cars.weight
X = sm.add_constant(X)
y = cars.mpg

model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.693
Model:                            OLS   Adj. R-squared:                  0.692
Method:                 Least Squares   F-statistic:                     878.8
Date:                Mon, 03 Oct 2016   Prob (F-statistic):          6.02e-102
Time:                        19:45:09   Log-Likelihood:                -1130.0
No. Observations:                 392   AIC:                             2264.
Df Residuals:                     390   BIC:                             2272.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         46.2165      0.799     57.867      0.000        44.646    47.787
weight        -0.0076      0.000    -29.645      0.000        -0.008    -0.007
==============================================================================
Omnibus:                       41.682   Durbin-Watson:                   0.808
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               60.039
Skew:                           0.727   Prob(JB):                     9.18e-14
Kurtosis:                       4.251   Cond. No.                     1.13e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.13e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

2) Comment on the model fit.  

The percentage of the variation in y (mpg) can be explained by the corresponding variation in X (weight) and the least-squares line is 69.3%, and the unexplained percentage of variation is 100% – 69.3% = 30.7%.

3) Interpret the model. 

Every unit of increase in weight affects the mpg by -0.0076.

4) Is `weight` useful for predicting `mpg`? Carry out a formal hypothesis test to show it.  

$H_0: \beta_1 = 0$  

$H_a: \beta_1 \neq 0$ 

Test statistic:  

$ t_{stat} = \frac{b_1 - 0}{s_{b_1}} = \frac{b_1}{s_{b_1}}$

In [5]:
#t_stats = results.params[1]/

5) Make a prediction for the average `mpg` of all cars that have a weight of 2000.  

In [6]:
46.2165 - 0.0076473 * 2000

30.9219

In [7]:
results.predict([1, 2000])

array([ 30.92183948])

6) Make a prediction for a particular car that has a weight of 2000.  

7) Write a Python function to calculate the confidence interval for your prediction in part 5).  

In [8]:
x = cars.weight
se = sqrt(results.mse_resid)
b0, b1 = results.params

x_new = 2000

def confidence_se(s_e, x, x_new):
    mean_x = x.mean()
    var_x = x.var()
    n = len(x)
    return s_e * (1/n + (x_new - mean_x)**2 / ((n - 1) * var_x))**0.5

sign = array([-1., 1.])
b0 + b1 * x_new + sign * 1.96 * confidence_se(se, x, x_new)

array([ 30.26741098,  31.57626797])

8) Write a Python function to calculate the prediction interval for your prediction in part 6).  

In [9]:
def prediction_se(s_e, x, x_new):
    mean_x = x.mean()
    var_x = x.var()
    n = len(x)
    return s_e * (1 + 1/n + (x_new - mean_x)**2 / ((n - 1) * var_x))**0.5

b0 + b1 * x_new + sign * 1.96 * prediction_se(se, x, x_new)

array([ 22.40454496,  39.439134  ])

9) What are the differences between the intervals you found in parts 7) and 8)?

**Question 2**  

You are shopping for a laptop computer at Best Buy. To help you with your decision, you decide to construct a regression model to predict the selling price of the laptop. The table `laptops` provides the following data for a random sample of laptops on Best Buy’s Web site:  

* Selling price
* Brand
* Screen size (in.)
* Hard drive size (GB)
* Amount of RAM memory (GB)
* Number of USB ports
* Weight (oz.) 

a) Using multiple regression, model selling price using the variables screen size, hard drive size, amount of ram, number of usb ports and weight.  

In [10]:
laptops = pd.read_sql("SELECT * FROM laptops AS l;", engine)

In [11]:
laptops.head()

,Price ($),Screen Size (in.),RAM Memory (GB),Hard drive (GB),USB Ports,Brand,Weight (oz.)
0,830,13.3,4,500,3,Toshiba,4.9
1,750,13.3,4,640,3,Toshiba,3.2
2,1200,11.6,2,128,2,Apple,2.3
3,1600,18.4,6,640,4,Toshiba,9.7
4,1900,18.4,8,500,4,Toshiba,9.7


In [12]:
# Lowercase and replace periods & spaces in the column names
new_names = []

for col in laptops.columns:
    new_names.append(col.replace('.', '', len(col)).replace(' ', '', len(col)).lower())

laptops.columns = new_names

print(laptops.columns)

Index(['price($)', 'screensize(in)', 'rammemory(gb)', 'harddrive(gb)',
       'usbports', 'brand', 'weight(oz)'],
      dtype='object')


In [13]:
X_multi = laptops[['screensize(in)', 'rammemory(gb)', 'harddrive(gb)', 'usbports', 'weight(oz)']]
X_multi = sm.add_constant(X_multi)
y_multi = laptops['price($)']

model_multi = sm.OLS(y_multi, X_multi)
results_multi = model_multi.fit()
results_multi.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               price($)   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     1.514
Date:                Mon, 03 Oct 2016   Prob (F-statistic):              0.200
Time:                        19:45:11   Log-Likelihood:                -477.99
No. Observations:                  63   AIC:                             968.0
Df Residuals:                      57   BIC:                             980.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
const            761.4987    946.954      0.804      0.425     -1134.744  2657.741
screensize(in)     4.1113     96.206      0.043      0.966      -188.539   196.761
rammemory(gb)     12.8642     74.411      0.173      0.863      -136.141   161.870
harddrive(gb)      0.6561      0.459      1.429      0.159        -0.263     1.576
usbports        -206.5346    123.563     -1.671      0.100      -453.965    40.896
weight(oz)        51.6251     99.210      0.520      0.605      -147.040   250.290
==============================================================================
Omnibus:                        9.835   Durbin-Watson:                   1.792
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               10.384
Skew:                           0.993   Prob(JB):                      0.00556
Kurtosis:                       3.102   Cond. No.                     8.40e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.4e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

b) Perform and interpret the overall F test.  

F-statistic:	1.514

Prob (F-statistic):	0.200

c) Using p-values, which variables appear to be needed in the model? Justify your answer.   

d) Now create a new predictor that contains random numbers drawn from your favorite distribution, and include this predictor in your multiple regression model. Comment on the model fit. How does the new $R^2$ compare to the one in part a)?  

In [14]:
mu, sigma = 0, 1
# We choose size equal to the size of the dataframe.
new_predictor = np.random.normal(mu, sigma, len(laptops))

In [15]:
laptops['new_predictor'] = new_predictor

In [16]:
laptops.head()

,price($),screensize(in),rammemory(gb),harddrive(gb),usbports,brand,weight(oz),new_predictor
0,830,13.3,4,500,3,Toshiba,4.9,2.374071
1,750,13.3,4,640,3,Toshiba,3.2,-0.594609
2,1200,11.6,2,128,2,Apple,2.3,0.872733
3,1600,18.4,6,640,4,Toshiba,9.7,-1.450908
4,1900,18.4,8,500,4,Toshiba,9.7,-1.241020


In [17]:
X_multi_new = laptops[['screensize(in)', 'rammemory(gb)', 'harddrive(gb)', 'usbports', 'weight(oz)', 'new_predictor']]
X_multi_new = sm.add_constant(X_multi_new)
y_multi_new = laptops['price($)']

model_multi_new = sm.OLS(y_multi_new, X_multi_new)
results_multi_new = model_multi_new.fit()
results_multi_new.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               price($)   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     1.242
Date:                Mon, 03 Oct 2016   Prob (F-statistic):              0.299
Time:                        19:45:11   Log-Likelihood:                -477.99
No. Observations:                  63   AIC:                             970.0
Df Residuals:                      56   BIC:                             985.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------------
const            773.8751    961.722      0.805      0.424     -1152.685  2700.435
screensize(in)     3.6470     97.140      0.038      0.970      -190.949   198.243
rammemory(gb)     11.4429     76.144      0.150      0.881      -141.091   163.977
harddrive(gb)      0.6629      0.467      1.419      0.162        -0.273     1.599
usbports        -207.7072    125.092     -1.660      0.102      -458.297    42.883
weight(oz)        51.7034    100.083      0.517      0.607      -148.788   252.195
new_predictor     -6.2666     56.328     -0.111      0.912      -119.104   106.571
==============================================================================
Omnibus:                       10.132   Durbin-Watson:                   1.793
Prob(Omnibus):                  0.006   Jarque-Bera (JB):               10.688
Skew:                           1.006   Prob(JB):                      0.00478
Kurtosis:                       3.152   Cond. No.                     8.45e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

e) Generate another new predictor - you can draw another list of random numbers from the same distribution as above, or you can draw from a different distribution. Add this predictor to the model in part d). What happends to the $R^2$? Does this mean that the new predictor is useful for predicting laptop prices?

In [18]:
lambda_predictor = 2
# We choose size equal to the size of the dataframe.
another_predictor = np.random.exponential(lambda_predictor, len(laptops))

In [19]:
laptops['another_predictor'] = another_predictor

In [20]:
laptops.head()

,price($),screensize(in),rammemory(gb),harddrive(gb),usbports,brand,weight(oz),new_predictor,another_predictor
0,830,13.3,4,500,3,Toshiba,4.9,2.374071,0.319790
1,750,13.3,4,640,3,Toshiba,3.2,-0.594609,10.704358
2,1200,11.6,2,128,2,Apple,2.3,0.872733,6.025138
3,1600,18.4,6,640,4,Toshiba,9.7,-1.450908,1.475372
4,1900,18.4,8,500,4,Toshiba,9.7,-1.241020,4.961939


In [21]:
X_multi_another = laptops[['screensize(in)', 'rammemory(gb)', 'harddrive(gb)', 'usbports', 'weight(oz)', 'new_predictor', 'another_predictor']]
X_multi_another = sm.add_constant(X_multi_another)
y_multi_another = laptops['price($)']

model_multi_another = sm.OLS(y_multi_another, X_multi_another)
results_multi_another = model_multi_another.fit()
results_multi_another.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               price($)   R-squared:                       0.135
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     1.223
Date:                Mon, 03 Oct 2016   Prob (F-statistic):              0.306
Time:                        19:45:11   Log-Likelihood:                -477.36
No. Observations:                  63   AIC:                             970.7
Df Residuals:                      55   BIC:                             987.9
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
const               771.4203    960.891      0.803      0.426     -1154.248  2697.089
screensize(in)       -4.1535     97.342     -0.043      0.966      -199.230   190.923
rammemory(gb)         8.0621     76.146      0.106      0.916      -144.538   160.662
harddrive(gb)         0.6684      0.467      1.432      0.158        -0.267     1.604
usbports           -237.9390    128.273     -1.855      0.069      -495.004    19.126
weight(oz)           77.5461    102.995      0.753      0.455      -128.861   283.953
new_predictor        -7.9644     56.302     -0.141      0.888      -120.797   104.868
another_predictor    34.5931     33.025      1.047      0.299       -31.591   100.778
==============================================================================
Omnibus:                       10.600   Durbin-Watson:                   1.745
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               10.987
Skew:                           1.010   Prob(JB):                      0.00411
Kurtosis:                       3.323   Cond. No.                     8.45e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

**Question 3**  

Squirt Squad is a cleaning service that sends crews to residential homes on either a once-a-month or twice-a-month schedule, depending on the customer’s preference. The owner would like to predict the amount of time required to clean a house based on the square footage of the house, the total number of rooms in the house, the number of bathrooms it has, the size of the cleaning crew, the frequency of the cleaning schedule, and whether or not the household has children. Data can be found in the tables **`squad`** (containing `squad_id`, `home_id`, `crew` and `freq` (0: once-a-month, 1: twice-a-month); **`squad_homes`** (containing `home_id`, `footage`, `rooms`, `baths` and `children` (Squirt Squad assumes the number of children in a house will never change. BONUS: how would you change the schema to account for the possibility that it will?)); and **`squad_times`** (containing `squad_id` and `dt`, `time` and `crew` (redundant with `squad` but included in case the squad size changes)). You will need to construct a three-way join using `home_id` and `squad_id`.

a) Construct a regression model using all of the independent variables.  

In [22]:
squad = pd.read_sql("SELECT s.squad_id, s.home_id, s.crew, s.freq, sh.footage,sh.rooms, sh.baths, sh.children, st.dt, st.time FROM squad AS s INNER JOIN squad_homes AS sh ON s.home_id = sh.home_id INNER JOIN squad_times AS st ON s.squad_id = st.squad_id;", engine)

In [23]:
squad.head()

,squad_id,home_id,crew,freq,footage,rooms,baths,children,dt,time
0,1,0,3,1,1548,8,2.0,0,2016-09-17,132
1,2,1,2,1,1599,7,1.5,0,2016-09-09,146
2,3,2,3,1,1630,8,2.0,0,2016-09-12,131
3,4,3,3,1,1640,7,1.5,0,2016-09-11,141
4,5,4,3,0,1711,8,2.5,1,2016-09-27,144


In [35]:
# We need to cast dt, which is of type datetime, to int to avoid error when running the OLS model.
squad['dt'] = squad['dt'].astype(int)

In [36]:
X_squad = squad[['crew', 'freq', 'footage', 'rooms', 'baths', 'children', 'dt']]
X_squad = sm.add_constant(X_squad)
y_squad = squad['time']

model_squad = sm.OLS(y_squad, X_squad)
results_squad = model_squad.fit()
results_squad.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   time   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                      -inf
Date:                Mon, 03 Oct 2016   Prob (F-statistic):                nan
Time:                        20:00:16   Log-Likelihood:                -1317.5
No. Observations:                 278   AIC:                             2637.
Df Residuals:                     277   BIC:                             2641.
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const       6.928e-35   7.68e-37     90.245      0.000      6.78e-35  7.08e-35
crew                0          0        nan        nan             0         0
freq        4.236e-35   4.69e-37     90.245      0.000      4.14e-35  4.33e-35
footage     1.655e-31   1.83e-33     90.245      0.000      1.62e-31  1.69e-31
rooms       7.548e-34   8.36e-36     90.245      0.000      7.38e-34  7.71e-34
baths       2.228e-34   2.47e-36     90.245      0.000      2.18e-34  2.28e-34
children    2.966e-35   3.29e-37     90.245      0.000       2.9e-35  3.03e-35
dt          1.019e-16   1.13e-18     90.245      0.000      9.97e-17  1.04e-16
==============================================================================
Omnibus:                        4.625   Durbin-Watson:                   1.729
Prob(Omnibus):                  0.099   Jarque-Bera (JB):                6.149
Skew:                           0.021   Prob(JB):                       0.0462
Kurtosis:                       3.727   Cond. No.                     1.12e+21
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.12e+21. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

b) Test and interpret the significance of the overall regression model (what is the result of the overall F test)?  

c) Interpret the meaning of the regression coefficient for the Rooms, Crew, Children, and Frequency variables.  

d) Using the p-values, identify which independent variables are significant (needed).  

e) Construct a regression model using only the significant variables found in part d) and predict the average time to clean a house that has 2,250 square feet, 11 total rooms, 3.5 bathrooms, and no children. This house is cleaned once a month with a crew of four employees.  

In [37]:
X_squad = squad[['crew', 'freq', 'footage', 'rooms', 'baths', 'children']]
X_squad = sm.add_constant(X_squad)
y_squad = squad['time']

model_squad = sm.OLS(y_squad, X_squad)
results_squad = model_squad.fit()
results_squad.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   time   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     44.51
Date:                Mon, 03 Oct 2016   Prob (F-statistic):           1.02e-37
Time:                        20:01:56   Log-Likelihood:                -1222.2
No. Observations:                 278   AIC:                             2458.
Df Residuals:                     271   BIC:                             2484.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const        143.5146     10.950     13.106      0.000       121.956   165.073
crew         -15.4141      2.026     -7.609      0.000       -19.402   -11.426
freq          -4.0902      2.621     -1.561      0.120        -9.250     1.070
footage       -0.0048      0.006     -0.772      0.441        -0.017     0.007
rooms          6.7580      1.238      5.457      0.000         4.320     9.196
baths         -5.6072      3.247     -1.727      0.085       -12.000     0.786
children      22.8734      2.618      8.738      0.000        17.720    28.027
==============================================================================
Omnibus:                        3.347   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.188   Jarque-Bera (JB):                3.313
Skew:                          -0.148   Prob(JB):                        0.191
Kurtosis:                       3.446   Cond. No.                     2.27e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.27e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

f) Compare the two models you fitted, which one is a better model? Why?